<a href="https://colab.research.google.com/github/skywalker00001/Conterfactual-Reasoning-Project/blob/main/time_travel1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
root = 'drive/MyDrive/LM/'

Mounted at /content/drive


In [ ]:
!pip install simplet5

     |████████████████████████████████| 1.2 MB 5.6 MB/s 
     |████████████████████████████████| 2.8 MB 50.4 MB/s 
     |████████████████████████████████| 919 kB 43.7 MB/s 
     |████████████████████████████████| 596 kB 38.1 MB/s 
     |████████████████████████████████| 132 kB 42.6 MB/s 
     |████████████████████████████████| 332 kB 54.5 MB/s 
     |████████████████████████████████| 829 kB 48.7 MB/s 
     |████████████████████████████████| 3.3 MB 39.9 MB/s 
     |████████████████████████████████| 61 kB 551 kB/s 
     |████████████████████████████████| 895 kB 38.6 MB/s 
     |████████████████████████████████| 1.1 MB 51.3 MB/s 
     |████████████████████████████████| 192 kB 56.9 MB/s 
     |████████████████████████████████| 271 kB 53.3 MB/s 
     |████████████████████████████████| 160 kB 53.3 MB/s 
  Created wheel for simplet5: filename=simplet5-0.1.3-py3-none-any.whl size=6831 sha256=5c35c089ddd61acf41907a9cc56e9f41bfcf650fdff72911c58097eee2078a05
  Stored in directory: /root/.cache/pi

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

path = root + '/TimeTravel/train_supervised_small.json'
df = pd.read_json(path, lines=True)
df.head()

,story_id,premise,initial,counterfactual,original_ending,edited_ending
0,080198fc-d0e7-42b3-8e63-b2144e59d816,On my way to work I stopped to get some coffee.,I went through the drive through and placed my...,I went inside to place my order.,I paid the cashier and patiently waited for my...,[I paid the cashier and patiently waited at th...
1,1ba02a18-8807-4f39-9271-ef555597ce21,Terry aspired to be a chef.,His father is one.,He moved to Italy and opened a restaurant.,He decided he would continue the business. He ...,"[He decided he would continue the business., H..."
2,dbb0ad3e-9389-44ee-8290-7c3458e3fa0f,Kim and her glass went on a field trip to an a...,Everyone enjoyed looking at the sea creatures.,Everyone did not enjoy looking at the sea cre...,"But when they went to the shark exhibit, Kim w...",[Instead of going home they went to the shark ...
3,e5955040-5b87-4acb-a8c7-7e81d0ffb9f5,Susie was sitting on her barstool.,She kept kicking the counter with her feet.,She kept herself steady with her feet.,"Suddenly, her kick sent her falling backwards....","[Suddenly, an earthquake sent her falling back..."
4,dc234072-2e69-4999-9e2f-632d3ea30b78,Celeste rode her motorcycle across the woods.,"When she almost arrived at the intersection, a...","When she almost arrived at the intersection, a...",She didn't have enough time to brake and drove...,[She didn't have enough time to brake and drov...


In [ ]:
print(df.loc[:,"edited_ending"][0:0])

Series([], Name: edited_ending, dtype: object)


In [ ]:
print(df.loc[:,"edited_ending"][0])

['I paid the cashier and patiently waited at the counter for my drink.', 'When she handed me the drink, the lid came off and spilled on me.', 'The coffee hurt and I had to go home and change clothes.']


In [ ]:
text_a, text_b = [], []

for i in range(len(df)):
  text_a.append("premise: " + df.loc[i, 'premise'] + " initial: " + df.loc[i, 'initial'] + " counterfactual: " + df.loc[i, 'counterfactual'] + " original_ending: " + df.loc[i, 'original_ending'])
  #text_a.append(re.sub(re_pat, df.loc[i, 'edit1'], df.loc[i, 'original1']))
  text_b.append("edited_ending: " + df.loc[i, 'edited_ending'][0] +" "+ df.loc[i, 'edited_ending'][1] +""+ df.loc[i, 'edited_ending'][2])

In [ ]:
processed_df = pd.DataFrame({'source_text': text_a, 'target_text': text_b}) 
processed_df.head()

,source_text,target_text
0,premise: On my way to work I stopped to get so...,edited_ending: I paid the cashier and patientl...
1,premise: Terry aspired to be a chef. initial: ...,edited_ending: He decided he would continue th...
2,premise: Kim and her glass went on a field tri...,edited_ending: Instead of going home they went...
3,premise: Susie was sitting on her barstool. in...,"edited_ending: Suddenly, an earthquake sent he..."
4,premise: Celeste rode her motorcycle across th...,edited_ending: She didn't have enough time to ...


In [24]:
print(processed_df.loc[0, "source_text"])
print("-------------")
print(processed_df.loc[0, "target_text"])

premise: On my way to work I stopped to get some coffee. initial: I went through the drive through and placed my order. counterfactual: I went inside to place my order. original_ending: I paid the cashier and patiently waited for my drink. When she handed me the drink, the lid came off and spilled on me. The coffee hurt and I had to go home and change clothes.
-------------
edited_ending: I paid the cashier and patiently waited at the counter for my drink. When she handed me the drink, the lid came off and spilled on me.The coffee hurt and I had to go home and change clothes.


In [ ]:
print(len(processed_df))

16752


In [ ]:
print(processed_df.shape)

(16752, 2)


In [ ]:
train_df, test_df = train_test_split(processed_df, test_size=0.2)
train_df.shape, test_df.shape

((13401, 2), (3351, 2))

In [11]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=test_df, 
            source_max_token_len=256, 
            target_max_token_len=128, 
            batch_size=8, max_epochs=3, use_gpu=True)

Global seed set to 42


Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

...

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [12]:
# let's load the trained model for inferencing:
model.load_model("t5","outputs/simplet5-epoch-2-train-loss-0.6946", use_gpu=True)

In [14]:
path = root + '/TimeTravel/train_unsupervised.json'
unsupervised_df = pd.read_json(path, lines=True)
unsupervised_df.head()

,story_id,premise,initial,counterfactual,original_ending
0,ece599cb-03d4-4856-b563-d4d4ed2f1424,The kids at school were discussing allowances.,Jeff was ashamed because he got less than anyo...,Jeff never had an allowance before.,"So, Jeff lied about how much allowance he got...."
1,d747fbd9-dae4-4dfe-a559-afc206d6ff94,Jason just needed a job.,He didn't care where it was or what he did.,He found a job as a waiter.,He applied to The Green Company. In the interv...
2,ef753e35-f541-4372-a2e9-1d11c6f6ad20,Kelly and her friends went to see a speaker in...,The girls were bored out of their minds.,The girls were listening intently to the speaker.,They had no idea what to do. Kelly ended up fa...
3,6fff7a67-933c-4c0e-a375-af6082ca275f,Andrew made a pizza.,"His roommate was very hungry, so he stole a sl...",He offered his roommate a slice but he declined.,His roommate discovered that the pizza had not...
4,13f6861b-14a6-49fb-ae63-ca154ab5a5d3,The couple argued over getting a real or a fak...,The got one of each to see how it looked in th...,The got a single real tree to see how it looke...,Both of them looked very similar at a distance...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [15]:
text = []

for i in range(len(df)):
  text.append("premise: " + df.loc[i, 'premise'] + " initial: " + df.loc[i, 'initial'] + " counterfactual: " + df.loc[i, 'counterfactual'] + " original_ending: " + df.loc[i, 'original_ending'])
  #text_a.append(re.sub(re_pat, df.loc[i, 'edit1'], df.loc[i, 'original1']))
  #text_b.append("edited_ending: " + df.loc[i, 'edited_ending'][0] +" "+ df.loc[i, 'edited_ending'][1] +""+ df.loc[i, 'edited_ending'][2])

In [16]:
processed_un_df = pd.DataFrame({'raw_text': text}) 
processed_un_df.head()

,raw_text
0,premise: On my way to work I stopped to get so...
1,premise: Terry aspired to be a chef. initial: ...
2,premise: Kim and her glass went on a field tri...
3,premise: Susie was sitting on her barstool. in...
4,premise: Celeste rode her motorcycle across th...


In [21]:

model.predict(processed_df.loc[3, 'source_text'])

['edited_ending: Suddenly, her kick sent her falling backwards. The chair hit the ground with a thud and broke.Susie hurt her head and was really scared.']